# TensorFlow를 사용한 ResNet Transfer Learning

이 노트북은 TensorFlow와 Keras를 사용하여 ImageNet pretrained 가중치를 가진 ResNet50 모델을 기반으로 Transfer Learning을 수행하는 예제입니다.

이미지는 지정된 디렉토리의 폴더 구조(ImageFolder 형식, 즉 클래스별 하위 폴더)를 사용하며, 학습/검증 데이터셋으로 분할됩니다.

모델은 ResNet50의 feature extractor 부분을 고정한 후, GlobalAveragePooling 및 Dense 레이어를 추가하여 이진 분류를 수행합니다.

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

### 1. 데이터셋 로드 및 전처리
# data_dir: 이미지가 폴더 구조로 저장되어 있는 경로 (예: '/path/to/images')
data_dir = '/path/to/images'  # 실제 이미지 폴더 경로로 변경하세요

# train/validation 데이터셋 생성 (80%/20% split)
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

# ResNet50에 맞는 전처리 함수 적용
from tensorflow.keras.applications.resnet50 import preprocess_input
AUTOTUNE = tf.data.AUTOTUNE

def preprocess(image, label):
    image = preprocess_input(image)
    return image, label

train_ds = train_ds.map(preprocess, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(preprocess, num_parallel_calls=AUTOTUNE)

### 2. Transfer Learning을 위한 ResNet50 모델 구성
# ImageNet 가중치를 사용한 ResNet50 (출력층 제외)
base_model = tf.keras.applications.ResNet50(
    weights='imagenet', 
    include_top=False, 
    input_shape=(224, 224, 3)
)
base_model.trainable = False  # base_model의 가중치는 고정

# 모델 구성: GlobalAveragePooling2D와 Dense 레이어 추가
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(2, activation='softmax')(x)  # 이진 분류 (클래스 2개)
model = tf.keras.Model(inputs, outputs)

# 모델 컴파일
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

### 3. 모델 학습
num_epochs = 10
history = model.fit(train_ds, epochs=num_epochs, validation_data=val_ds)

### 4. 성능 지표 평가 (Accuracy, Recall, Precision, F1 Score)
y_true = []
y_pred = []
for images, labels in val_ds:
    preds = model.predict(images)
    preds = np.argmax(preds, axis=1)
    y_pred.extend(preds)
    y_true.extend(labels.numpy())

print('Accuracy:', accuracy_score(y_true, y_pred))
print('Recall:', recall_score(y_true, y_pred, average='weighted'))
print('Precision:', precision_score(y_true, y_pred, average='weighted'))
print('F1 Score:', f1_score(y_true, y_pred, average='weighted'))